In [14]:
# 🔄 Install dependencies (if using Google Colab)
!pip install google-generativeai tavily-python

In [15]:
import google.generativeai as genai
from tavily import TavilyClient
import json
from datetime import datetime

In [16]:
!pip install python-dotenv

In [17]:
from dotenv import load_dotenv
import os

# 🔄 Load custom .env file (not named .env by default)
load_dotenv(dotenv_path="API_Key.env")

# 🔐 Fetch keys
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

# ✅ Check and display partial key
if not GEMINI_API_KEY:
    raise ValueError("❌ GEMINI_API_KEY not found in API_Key.env")

if not TAVILY_API_KEY:
    raise ValueError("❌ TAVILY_API_KEY not found in API_Key.env")

print("✅ Gemini API Key Loaded:", GEMINI_API_KEY[:5] + "*****")
print("✅ Tavily API Key Loaded:", TAVILY_API_KEY[:5] + "*****")

✅ Gemini API Key Loaded: AIzaS*****
✅ Tavily API Key Loaded: tvly-*****


In [18]:
# 🔐 Authenticate with Google Generative AI
genai.configure(api_key=GEMINI_API_KEY)

# 🔐 Authenticate Tavily
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

In [19]:
# initilize the clinets - api module
model = genai.GenerativeModel('gemini-2.0-flash')

In [20]:
def fact_check(claim):
    # Step 1: Print the claim being fact-checked
    print(f"Fact-Checking: {claim}")
    print(f"="*60)

    try:
        # Step 2: Use Tavily API to search for evidence from the web
        print(f"Searching for evidence....")
        search_result = tavily_client.search(
            query=claim,                # The claim to search
            search_depth="advanced",   # Depth of search ("basic" or "advanced")
            max_results=5              # Max number of results to fetch
        )

        # Step 3: If no results are found, notify the user
        if not search_result.get('results'):
            print(f"No search results were found....")

        # Initialize empty evidence text and sources list
        evidence = ""
        sources = []

        # Step 4: Loop through top 5 results and collect title, snippet, and URL
        for i, result in enumerate(search_result.get('results', [])[:5]):
            title = result.get('title', "Unknown")       # Title of the source
            content = result.get('content', "No content")# Content snippet
            url = result.get('url', "NO URL")            # URL to the article

            # Append content snippet to evidence text
            evidence += f"Source : {i+1}:{title}\n{content[:400]}....\n\n"

            # Save source for display later
            sources.append({'title': title, 'url': url})

        print(f"Found {len(sources)} Sources")

        # Step 5: Use Gemini model to analyze the evidence
        print(f"Analyzing claim.....")

        # Create prompt for Gemini to evaluate claim
        prompt = f"""
        You are a professional fact-checker. Analyze this claim against the evidence:

        CLAIM:{claim}

        EVIDENCE:{evidence}

        Provide a fact-check analysis with:

        1. VERDICT: Choose ONE of these:
            -TRUE: Claim is factually accurate
            -FALSE: Claim is factually incorrect
            -PARTIALLY TRUE: Claim has some truth but is misleading
            -UNVERIFIED: Insufficient evidence to determine accuracy
            -OUTDATED: Claim was true but circumstances have changed

        2. CONFIDENCE: High/Medium/Low

        3. EXPLANATION: Clear 2-3 lines sentence explanation of your verdict

        4. KEY EVIDENCE: Brief summary of the most important evidence

        keep it concise and factual
        """

        # Generate content using the Gemini model
        response = model.generate_content(prompt)
        analysis = response.text  # Extract text response

        # Step 6: Display the analysis results
        print(f"FACT CHECK RESULTS....")
        print("="*60)
        print(analysis)

        # Step 7: Print all the sources used
        print(f"SOURCES.....")
        print(f"-"*30)
        for i, source in enumerate(sources, 1):
            print(f"{i}. {source['title']}")
            print(f"  {source['url']}")

        # Final step: Show timestamp
        print(f"Completed at: {datetime.now()}")

    except Exception as e:
        # Catch and print any unexpected errors
        print(f"Error:{str(e)}")

In [21]:
def main():
    print("SIMPLE FACT CHECKER WITH GEMINI AND TAVILY")
    print("Type 'quit' to exit")
    print("="*60)

    while True:
        try:
            # Prompt the user to input a claim
            claim = input("\n Enter your claim to fact-check: ").strip()

            # Exit condition
            if claim.lower() in ['quit', 'q', 'exit']:
                print("Goodbye")
                break

            # If user enters nothing, prompt again
            if not claim:
                print(f"Please enter the claim")
                continue

            # Call the fact-check function
            fact_check(claim)
            print("\n" + "="*60)

        except KeyboardInterrupt:
            # Graceful exit on Ctrl+C
            print("Goodbye")
            break
        except Exception as e:
            # Catch and show unexpected errors
            print(f"Unexpected error: {str(e)}")

In [22]:
if __name__ == "__main__":
    main()  # Run the main function only when the script is run directly

SIMPLE FACT CHECKER WITH GEMINI AND TAVILY
Type 'quit' to exit

 Enter your claim to fact-check: it is hard for a fresher to enter data science jobs.
Fact-Checking: it is hard for a fresher to enter data science jobs.
Searching for evidence....
Found 5 Sources
Analyzing claim.....
FACT CHECK RESULTS....
1.  VERDICT: PARTIALLY TRUE
2.  CONFIDENCE: Medium
3.  EXPLANATION: While entering data science as a fresher can be challenging due to required skills and education, it is not impossible, as evidenced by suggestions for overcoming these challenges through mentorship, targeted skill development, and gaining relevant experience. The need for specific degrees or experience indicates a barrier to entry.
4.  KEY EVIDENCE: Sources 1, 2, 3, and 4 indicate the need for specific skills, education, or experience to enter the data science field, but also offer advice on how to overcome these challenges.

SOURCES.....
------------------------------
1. Overcoming Challenges in Data Science Careers
 